In [27]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

In [ ]:
# อ่านไฟล์ CSV
file_path = 'sit-stand-walk-sleep.csv'  # กำหนดไฟล์ CSV
df = pd.read_csv(file_path)  # อ่านข้อมูลจากไฟล์ CSV

# แสดง DataFrame ในรูปแบบตาราง
df

In [ ]:
# โฟลเดอร์สำหรับบันทึกภาพกราฟแต่ละเฟรม
output_dir = 'output_frames'
os.makedirs(output_dir, exist_ok=True)

plt.switch_backend('Agg')

# ฟังก์ชันสำหรับการสร้าง skeleton พร้อม grid
def plot_skeleton(frame_data, frame_number):
    # กำหนดจุด landmark ที่จะเชื่อมต่อกัน
    connections = [
        (0, 1), (1, 2), (2, 3), (3, 7), (0, 4), (4, 5), (5, 6), (6, 8), (9, 10),  # หัว
        (11, 12), (12, 24), (24, 23), (23, 11),  # ลำตัว
        (11, 13), (13, 15), (12, 14), (14, 16), # แขน
        (15, 17), (15, 21), (15, 19), (19, 17), # มือซ้าย
        (16, 18), (16, 22), (16, 20), (20, 18), # มือขวา
        (23, 25), (25, 27), (27, 29), (29, 31), (31, 27),  # ขาขวา
        (24, 26), (26, 28), (28, 30), (30, 32), (28, 32)  # ขาซ้าย
    ]
    
    # ดึงค่าข้อมูล x และ y สำหรับแต่ละ landmark
    x_vals = np.array([frame_data[f'x{i}'] for i in range(33)])
    y_vals = np.array([frame_data[f'y{i}'] for i in range(33)])
    z_vals = np.array([frame_data[f'z{i}'] for i in range(33)])
    
    # สร้างภาพแบบ subplot ที่มี 2D และ 3D
    fig = plt.figure(figsize=(15, 5))
    
    # วาด 2D skeleton ใน subplot แรก
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.set_title(f'2D Poses at Frame {frame_number}')
    ax1.scatter(x_vals, y_vals, c='green', s=40)  # วาดจุด
    for conn in connections:
        ax1.plot([x_vals[conn[0]], x_vals[conn[1]]], [y_vals[conn[0]], y_vals[conn[1]]], 'r-')  # วาดเส้นเชื่อมต่อ
    ax1.grid(True)  # เปิดกริด
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    
    # กำหนดขนาดสูงสุดของแกน
    ax1.set_xlim(0, 1920)  # ตั้งค่าขนาดแกน x
    ax1.set_ylim(1080, 0)  # ตั้งค่าขนาดแกน y

    # บันทึกภาพเป็นไฟล์ PNG    
    # plt.savefig(f"output_frames/{frame_number}")

    # วาด 3D skeleton ใน subplot ที่สอง
    ax2 = fig.add_subplot(1, 2, 2, projection='3d')
    ax2.set_title(f'3D Poses at Frame {frame_number}')
    
    ax2.scatter(z_vals, x_vals, y_vals, c='green', s=3)  # สลับแกน X กับ Z

    for conn in connections:
        ax2.plot(
                [z_vals[conn[0]], z_vals[conn[1]]],
                [x_vals[conn[0]], x_vals[conn[1]]], 
                [y_vals[conn[0]], y_vals[conn[1]]],
                'r-')

    # ตั้งชื่อแกนและกริด
    ax2.set_xlabel('Z')  # ตอนนี้แกน X เดิมคือ Z
    ax2.set_ylabel('X')  # แกน Y คงเดิม
    ax2.set_zlabel('Y')  # ตอนนี้แกน Z เดิมคือ X
    ax2.grid(True)  # เปิดกริด

    # กำหนดขนาดสูงสุดของแกน
    ax2.set_xlim(1, -1)  # ตั้งค่าขนาดแกน x
    ax2.set_ylim(0, 1920)  # ตั้งค่าขนาดแกน y
    ax2.set_zlim(1080, 0)  # ตั้งค่าขนาดแกน y

    # บันทึกภาพเป็นไฟล์ PNG    
    plt.savefig(f"{output_dir}/frame_{frame_number}.png")
    # plt.show()
    plt.close()

# ฟังก์ชันหลักที่ใช้สำหรับการประมวลผลแบบขนาน
# def process_frames(selected_frames):
#     for frame in selected_frames:
#         frame_data = df.loc[df['frames'] == frame].squeeze()
#         plot_skeleton(frame_data, frame)

# ฟังก์ชันที่ใช้สำหรับ ProcessPoolExecutor
def process_single_frame(frame):
    frame_data = df.loc[df['frames'] == frame].squeeze()
    plot_skeleton(frame_data, frame)
    return 1  # ส่งค่ากลับมาเพื่อบอกจำนวนเฟรมที่ประมวลผล

# แสดง skeleton สำหรับเฟรมที่เลือก
selected_frames = df['frames'].tolist()
# selected_frames = [200, 450, 700, 875, 900]

# ใช้ ThreadPoolExecutor เพื่อประมวลผลเฟรมแบบขนาน
# with ThreadPoolExecutor(max_workers=9) as executor:
#     futures = [executor.submit(process_frames, [frame]) for frame in selected_frames]

# # รอให้ทุกฟังก์ชันทำงานเสร็จ
# for future in futures:
#     future.result()  # ตรวจสอบว่ามีข้อผิดพลาดหรือไม่
    

count = 0
# ใช้ ProcessPoolExecutor เพื่อประมวลผลเฟรมแบบขนาน
with ProcessPoolExecutor(max_workers=2) as executor:
    results = list(executor.map(process_single_frame, selected_frames))
    count = sum(results)  # นับจำนวนเฟรมที่ประมวลผล

# num_cores = os.cpu_count()
# print(f"Number of CPU cores: {num_cores}")

print(count)  # พิมพ์จำนวนเฟรมที่ประมวลผล
print("บันทึกภาพเสร็จสิ้น")

In [23]:
# สร้างวิดีโอจากภาพที่บันทึกไว้โดยใช้ OpenCV
video_path = 'output_video_V2.mp4'
frame_rate = 60  # กำหนด frame rate ของวิดีโอ
frame_size = (1920, 1080)  # ขนาดเฟรม (กว้าง x สูง)

# ใช้ OpenCV ในการสร้างวิดีโอ
out = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'mp4v'), frame_rate, frame_size)

# อ่านภาพที่บันทึกไว้และใส่ลงในวิดีโอ
for frame in selected_frames:
    img_path = os.path.join(output_dir, f'{frame}.png')
    img = cv2.imread(img_path)
    
    # ปรับขนาดภาพถ้าจำเป็น
    img = cv2.resize(img, frame_size)
    
    # ใส่ภาพเข้าไปในวิดีโอ
    out.write(img)

# ปิดการเขียนวิดีโอ
out.release()

print(f'Video saved as {video_path}')